```
This software is part of GPU Ocean. 

Copyright (C) 2018 SINTEF Digital
Copyright (C) 2018 Norwegian Meteorological Institute

This notebook implements the experiments to evaluate convergence rates
for the four numerical schemes, as reported in Section 4.7 in Test Cases for 
Rotational Shallow-Water Schemes by Holm, Brodtkorb, Broström, Christensen 
and Sætra. 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Numerical Order

This notebook runs an experiment to evaluate the numerical order for each of the four schemes. By running this notebook, the convergence plots presented in Section 4.7 "Numerical Order" in the paper should be reproduced.



## Import modules and set up environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt

import os
import datetime
import gc
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

#Set large figure sizes
rc('figure', figsize=(6.0, 4.0))
rc('animation', html='html5')

#Import our simulator
from SWESimulators import FBL, CTCS,KP07, CDKLM16, PlotHelper, Common, IPythonMagic
#Import initial condition and bathymetry generating functions:
from SWESimulators.BathymetryAndICs import *

In [ ]:
%setup_logging compareschemes2d.log
%cuda_context_handler gpu_ctx

In [ ]:
#Create output directory for images
imgdir='images_convergence_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print("Saving images to " + imgdir)

In [ ]:
def setBwStyles(ax):
    from cycler import cycler

    ax.set_prop_cycle( cycler('marker', ['.', 'x', 4, '+', '*', '1']) +
                       cycler('linestyle', ['-.', '--', ':', '-.', '--', ':']) +
                       #cycler('markersize', [15, 15, 15, 15, 15, 15]) +
                       cycler('color', ['k', 'k', 'k', 'k', 'k', 'k']) )

In [ ]:
width = 100
height = 100

domain_min = 16
    
g = 9.81
r = 0.0
#f = 8.0e-5
f = 0.0

end_time = 1

In [ ]:
def average(highres_solution, factor):
    NX = highres_solution.shape[1]
    NY = highres_solution.shape[0]
    
    assert(NY % factor == 0)
    assert(NX % factor == 0)
    
    nx = int(NX / factor)
    ny = int(NY / factor)
    
    factor = int(factor)
    
    return  highres_solution.reshape([ny, factor, nx, factor]).mean(3).mean(1)

In [ ]:
def initH(h0, ghosts):    
    waterHeight = 0.5
    h0.fill(waterHeight)
    
def initEtaFV(eta0, ghosts, reference_domain_size):
    ref_nx = reference_domain_size
    ref_ny = reference_domain_size
    nx = eta0.shape[1] - ghosts[0] - ghosts[2]
    ny = eta0.shape[0] - ghosts[1] - ghosts[3]
    
    def my_cos(i, j):
        size = 0.6
        x = 2*(i + 0.5 - ref_nx/2.0) / float(ref_nx)
        y = 2*(j + 0.5 - ref_ny/2.0) / float(ref_ny)
        r = np.sqrt(x**2 + y**2)
        return 0.05*(1.0 + np.cos(np.pi*r/size)) * (r < size)
    
    #Generate disturbance 
    disturbance = np.fromfunction(lambda i, j: my_cos(i,j), (ref_ny, ref_nx))
    disturbance = average(disturbance, ref_nx/nx) 
    
    eta0.fill(0.0)
    x0, x1 = ghosts[0], nx+ghosts[0]
    y0, y1 = ghosts[1], ny+ghosts[1]
    eta0[y0:y1, x0:x1] += (0.01*disturbance)
    

def initEtaFD(eta0, ghosts):
    nx = eta0.shape[1] - ghosts[0] - ghosts[2]
    ny = eta0.shape[0] - ghosts[1] - ghosts[3]
    
    def my_cos(i, j):
        size = 0.6
        x = 2*(i - (nx-1)/2.0) / float(nx-1)
        y = 2*(j - (ny-1)/2.0) / float(ny-1)
        r = np.sqrt(x**2 + y**2)
        return 0.5*(1.0 + np.cos(np.pi*r/size)) * (r < size)
    
    #Generate disturbance 
    disturbance = np.fromfunction(lambda i, j: my_cos(i,j), (ny, nx))
    
    eta0.fill(0.0)
    x0, x1 = ghosts[0], nx+ghosts[0]
    y0, y1 = ghosts[1], ny+ghosts[1]
    eta0[y0:y1, x0:x1] += (0.01*disturbance)
    
    
def initHU(hu0, ghosts):
    hu0.fill(0.0)
    
def initHV(hv0, ghosts):
    hv0.fill(0.0)

def testInitDataFV(domain_size, reference_domain_size):
    
    nx = domain_size
    ny = domain_size
    
    ghosts = [1, 1, 1, 1] 
    dataShape = (ny + ghosts[1]+ghosts[3], 
                 nx + ghosts[0]+ghosts[2])

    eta0 = np.zeros(dataShape, dtype=np.float32);
    
    initEtaFV(eta0, ghosts, reference_domain_size)
    
    return eta0#[ghosts[1]:eta0.shape[1]+ghosts[1], ghosts[0]:eta0.shape[0]+ghosts[0]]

def testInitDataFD(domain_size):
    nx = domain_size
    ny = domain_size
    
    ghosts = [1, 1, 1, 1] 
    dataShape = (ny + ghosts[1]+ghosts[3], 
                 nx + ghosts[0]+ghosts[2])

    eta0 = np.zeros(dataShape, dtype=np.float32);
    
    initEtaFD(eta0, ghosts)
    
    return eta0#[ghosts[1]:eta0.shape[1]+ghosts[1], ghosts[0]:eta0.shape[0]+ghosts[0]]
    
plt.figure(figsize=(12,8))
print("Finite volume")
for i in range(5):
    domain_size = domain_min*2**i
    eta0 = testInitDataFV(domain_size, domain_min*2**6)
    plt.subplot(1, 6, i+1)
    plt.imshow(eta0, interpolation='nearest', origin='lower')
    plt.title("FV"+str(domain_size))
    print("FV: Max={:.05f}, min={:.05f}, sum={:.010f}".format(np.max(eta0), np.min(eta0), np.sum(eta0/(domain_size*domain_size))))
    
plt.figure(figsize=(12,8))
print("Finite difference")
domain_size = int(domain_min/2)
for i in range(5):
    domain_size = domain_size*2-1
    eta0 = testInitDataFD(domain_size)
    plt.subplot(1, 6, i+1)
    plt.imshow(eta0, interpolation='nearest', origin='lower')
    plt.title("FD"+str(domain_size))
    print("FD: Max={:.05f}, min={:.05f}, sum={:.010f}".format(np.max(eta0), np.min(eta0), np.sum(eta0/(domain_size*domain_size))))

In [ ]:
def plotData(eta0, u0, v0, eta1, u1, v1):
    fig, axarr = plt.subplots(2, 3, figsize=(12, 8))
    axarr[0, 0].imshow(eta0, interpolation="nearest")
    axarr[0, 1].imshow(u0, interpolation="nearest")
    axarr[0, 2].imshow(v0, interpolation="nearest")
    axarr[1, 0].imshow(eta1, interpolation="nearest")
    axarr[1, 1].imshow(u1, interpolation="nearest")
    axarr[1, 2].imshow(v1, interpolation="nearest")
    print("Eta0: Maximum = {:.05f}, minimum = {:.05f}".format(np.max(eta0), np.min(eta0)))
    print("Eta1: Maximum = {:.05f}, minimum = {:.05f}".format(np.max(eta1), np.min(eta1)))

## Forward Backward Linear

In [ ]:
def benchmarkFBL(fbl_filename):
    def runFBL(domain_size):
        #Clean up old simulator if any:
        if 'fbl_sim' in globals():
            fbl_sim.cleanUp()

        nx = domain_size
        ny = domain_size

        dx = float(width/(nx-1))
        dy = float(height/(ny-1))

        ghosts = [0, 0, 0, 0]
        dataShape = (ny + ghosts[1]+ghosts[3], 
                     nx + ghosts[0]+ghosts[2])

        h0 = np.zeros(dataShape, dtype=np.float32);
        eta0 = np.zeros(dataShape, dtype=np.float32);
        hu0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
        hv0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);
        
        max_h_estimate = 0.6
        max_u_estimate = 0.1*np.sqrt(2.0)
        dt = 0.45 * min(dx, dy) / (max_u_estimate + np.sqrt(g*max_h_estimate))

        initH(h0, ghosts)
        initEtaFD(eta0, ghosts)
        initHU(hu0, ghosts)
        initHV(hv0, ghosts)

        #Initialize simulator
        fbl_sim = FBL.FBL(gpu_ctx, \
                      h0, eta0, hu0, hv0, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r)

        t = fbl_sim.step(end_time)
        eta1, hu1, hv1 = fbl_sim.download()
        print("Domain size={:d}, dt={:f}, nt={}, eta=[{:f}, {:f}]".format(domain_size, dt, fbl_sim.num_iterations, np.min(eta1), np.max(eta1)))

        return [eta0, hu0, hv0, eta1, hu1, hv1]

    #Run all domain sizes 
    data = {};
    domain_size = int((domain_min)/2)
    for i in range(9):
        domain_size = domain_size*2-1
        [_, _, _, eta1, _, _] = runFBL(domain_size)
        data[str(domain_size)] = eta1

    #Save to file
    np.savez_compressed(fbl_filename, **data)
    
fbl_filename = imgdir + "/FBL_data.npz"
benchmarkFBL(fbl_filename)

In [ ]:
domain_sizes = []
error = []

with np.load(fbl_filename) as npzfile:
    #Get all domain sizes
    domain_sizes = np.array(npzfile.keys(), dtype=np.int)
    error = np.zeros(len(domain_sizes)-1)
    
    #Get largest domain size
    reference_domain_size = np.max(domain_sizes)
    
    #Remove largest from all domain sizes
    domain_sizes = np.delete(domain_sizes, np.where(domain_sizes == reference_domain_size))
    
    #Get reference
    eta1_ref_disk = npzfile[str(reference_domain_size)].astype(np.float64)
    
    #Loop over all other domain sizes
    for l, domain_size in enumerate(domain_sizes):
        #Get simulation
        eta1 = npzfile[str(domain_size)].astype(np.float64)
        
        #Downsample reference to simulation size
        eta1_ref = eta1_ref_disk.copy()
        while (eta1_ref.shape[0] > eta1.shape[0]):
            eta1_ref = eta1_ref[::2,::2]
        
        plt.figure()
        plt.imshow(eta1 - eta1_ref)
        plt.colorbar()
        #eta1_ref_downsampled = eta1_ref[::downsampling, ::downsampling]
        error[l] = np.linalg.norm((eta1_ref - eta1).flatten(), ord=1) / (domain_size*domain_size)
        print(error[l])
        
fig = plt.figure(figsize=(12, 8))
setBwStyles(fig.gca())
plt.loglog(domain_sizes, error)
plt.gca().axis('equal')

## Centered in time, centered in space

In [ ]:
def benchmarkCTCS(ctcs_filename):
    
    def runCTCS(domain_size):
        #Clean up old simulator if any:
        if 'ctcs_sim' in globals():
            ctcs_sim.cleanUp()

        nx = domain_size
        ny = domain_size

        dx = float(width/(nx-1))
        dy = float(height/(ny-1))

        ghosts = [1, 1, 1, 1]
        dataShape = (ny + ghosts[1]+ghosts[3], 
                     nx + ghosts[0]+ghosts[2])

        h0 = np.zeros(dataShape, dtype=np.float32);
        eta0 = np.zeros(dataShape, dtype=np.float32);
        hu0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32);
        hv0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32);
        
        max_h_estimate = 0.6
        max_u_estimate = 0.1*np.sqrt(2.0)
        dt = 0.15 * min(dx, dy) / (max_u_estimate + np.sqrt(g*max_h_estimate))

        initH(h0, ghosts)
        initEtaFD(eta0, ghosts)
        initHU(hu0, ghosts)
        initHV(hv0, ghosts)

        A = 0.0

        #Initialize simulator
        ctcs_sim = CTCS.CTCS(gpu_ctx, \
                      h0, eta0, hu0, hv0, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r, A)

        t = ctcs_sim.step(end_time)
        eta1, hu1, hv1 = ctcs_sim.download()
        print("Domain size={:d}, t={:f}, nt={}, eta=[{:f}, {:f}]".format(domain_size, t, ctcs_sim.num_iterations, np.min(eta1), np.max(eta1)))

        return [eta0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                eta1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]]]
        #return [eta0, hu0, hv0, eta1, hu1, hv1]


    #Run all domain sizes 
    data = {};
    domain_size = int((domain_min)/2)
    for i in range(9):
        domain_size = domain_size*2-1
        [_, _, _, eta1, _, _] = runCTCS(domain_size)
        data[str(domain_size)] = eta1
        #plt.figure()
        #plt.imshow(eta1)

    #Save to file
    np.savez_compressed(ctcs_filename, **data)
    
    
ctcs_filename = imgdir + "/CTCS_data.npz"
benchmarkCTCS(ctcs_filename)

In [ ]:
domain_sizes = []
error = []

with np.load(ctcs_filename) as npzfile:
    #Get all domain sizes
    domain_sizes = np.array(npzfile.keys(), dtype=np.int)
    error = np.zeros(len(domain_sizes)-1)
    
    #Get largest domain size
    reference_domain_size = np.max(domain_sizes)
    
    #Remove largest from all domain sizes
    domain_sizes = np.delete(domain_sizes, np.where(domain_sizes == reference_domain_size))
    
    #Get reference
    eta1_ref_disk = npzfile[str(reference_domain_size)].astype(np.float64)
    
    #Loop over all other domain sizes
    for l, domain_size in enumerate(domain_sizes):
        #Get simulation
        eta1 = npzfile[str(domain_size)].astype(np.float64)
        
        #Downsample reference to simulation size
        eta1_ref = eta1_ref_disk.copy()
        while (eta1_ref.shape[0] > eta1.shape[0]):
            eta1_ref = eta1_ref[::2,::2]
        
        plt.figure()
        plt.imshow(eta1 - eta1_ref)
        plt.colorbar()
        #eta1_ref_downsampled = eta1_ref[::downsampling, ::downsampling]
        error[l] = np.linalg.norm((eta1_ref - eta1).flatten(), ord=1) / (domain_size*domain_size)
        print(error[l])
        
fig = plt.figure(figsize=(12, 8))
setBwStyles(fig.gca())
plt.loglog(domain_sizes, error)
plt.gca().axis('equal')

## CDKLM 16

In [ ]:
def benchmarkCDKLM(cdklm_filename):
    
    def runCDKLM(domain_size, reference_domain_size):
        #Clean up old simulator if any:
        if 'cdklm_sim' in globals():
            cdklm_sim.cleanUp()

        nx = domain_size
        ny = domain_size

        dx = float(width/nx)
        dy = float(height/ny)

        ghosts = np.array([2,2,2,2])
        dataShape = (ny + ghosts[1]+ghosts[3], 
                     nx + ghosts[0]+ghosts[2])

        Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32)
        eta0 = np.zeros(dataShape, dtype=np.float32);
        hu0 = np.zeros(dataShape, dtype=np.float32);
        hv0 = np.zeros(dataShape, dtype=np.float32);
        
        max_h_estimate = 0.6
        max_u_estimate = 0.1*np.sqrt(2.0)
        dt = 0.45 * min(dx, dy) / (max_u_estimate + np.sqrt(g*max_h_estimate))

        initH(Hi, ghosts)
        initEtaFV(eta0, ghosts, reference_domain_size)
        initHU(hu0, ghosts)
        initHV(hv0, ghosts)

        #Initialize simulator
        cdklm_sim = CDKLM16.CDKLM16(gpu_ctx, \
                      eta0, hu0, hv0, Hi, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r)

        t = cdklm_sim.step(end_time)
        eta1, hu1, hv1 = cdklm_sim.download()
        print("Domain size={:d}, t={:f}, nt={}, eta=[{:f}, {:f}]".format(domain_size, t, cdklm_sim.num_iterations, np.min(eta1), np.max(eta1)))

        return [eta0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                eta1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]]]
        #return [eta0, hu0, hv0, eta1, hu1, hv1]


    #Run all domain sizes 
    data = {};
    domain_size = int((domain_min)/2)
    num_levels=7
    reference_domain_size = domain_min*2**num_levels
    for i in range(num_levels+1):
        domain_size = domain_min*2**i
        [_, _, _, eta1, _, _] = runCDKLM(domain_size, reference_domain_size)
        data[str(domain_size)] = eta1

    #Save to file
    np.savez_compressed(cdklm_filename, **data)
    
    
cdklm_filename = imgdir + "/CDKLM_data.npz"
benchmarkCDKLM(cdklm_filename)

In [ ]:
domain_sizes = []
error = []

with np.load(cdklm_filename) as npzfile:
    #Get all domain sizes
    domain_sizes = np.array(npzfile.keys(), dtype=np.int)
    error = np.zeros(len(domain_sizes)-1)
    
    #Get largest domain size
    reference_domain_size = np.max(domain_sizes)
    
    #Remove largest from all domain sizes
    domain_sizes = np.delete(domain_sizes, np.where(domain_sizes == reference_domain_size))
    
    #Get reference
    eta1_ref_disk = npzfile[str(reference_domain_size)].astype(np.float64)
    print(eta1_ref_disk.shape)
    
    #Loop over all other domain sizes
    for l, domain_size in enumerate(domain_sizes):
        #Get simulation
        print(domain_size)
        eta1 = npzfile[str(domain_size)].astype(np.float64)
        
        #Downsample reference to simulation size
        eta1_ref = average(eta1_ref_disk.copy(), reference_domain_size/domain_size)
        
        plt.figure()
        plt.imshow(eta1 - eta1_ref)
        plt.colorbar()
        #eta1_ref_downsampled = eta1_ref[::downsampling, ::downsampling]
        error[l] = np.linalg.norm((eta1_ref - eta1).flatten(), ord=1) / (domain_size*domain_size)
        print(error[l])
        
fig = plt.figure(figsize=(12, 8))
setBwStyles(fig.gca())
plt.loglog(domain_sizes, error)
plt.gca().axis('equal')

## Kurganov-Petrova 2007

In [ ]:
def benchmarkKP07(cdklm_filename):
    
    def runKP07(domain_size, reference_domain_size):
        #Clean up old simulator if any:
        if 'kp07_sim' in globals():
            kp07_sim.cleanUp()

        nx = domain_size
        ny = domain_size

        dx = float(width/nx)
        dy = float(height/ny)

        ghosts = np.array([2,2,2,2])
        dataShape = (ny + ghosts[1]+ghosts[3], 
                     nx + ghosts[0]+ghosts[2])

        Hi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32)
        eta0 = np.zeros(dataShape, dtype=np.float32);
        hu0 = np.zeros(dataShape, dtype=np.float32);
        hv0 = np.zeros(dataShape, dtype=np.float32);
        
        max_h_estimate = 0.6
        max_u_estimate = 0.1*np.sqrt(2.0)
        dt = 0.45 * min(dx, dy) / (max_u_estimate + np.sqrt(g*max_h_estimate))

        initH(Hi, ghosts)
        initEtaFV(eta0, ghosts, reference_domain_size)
        initHU(hu0, ghosts)
        initHV(hv0, ghosts)

        #Initialize simulator
        kp07_sim = KP07.KP07(gpu_ctx, \
                      eta0, hu0, hv0, Hi, \
                      nx, ny, \
                      dx, dy, dt, \
                      g, f, r)

        t = kp07_sim.step(end_time)
        eta1, hu1, hv1 = kp07_sim.download()
        print("Domain size={:d}, t={:f}, nt={}, eta=[{:f}, {:f}]".format(domain_size, t, kp07_sim.num_iterations, np.min(eta1), np.max(eta1)))

        return [eta0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv0[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                eta1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hu1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]], 
                hv1[ghosts[1]:-ghosts[3], ghosts[0]:-ghosts[2]]]
        #return [eta0, hu0, hv0, eta1, hu1, hv1]


    #Run all domain sizes 
    data = {};
    domain_size = int((domain_min)/2)
    num_levels=7
    reference_domain_size = domain_min*2**num_levels
    for i in range(num_levels+1):
        domain_size = domain_min*2**i
        [_, _, _, eta1, _, _] = runKP07(domain_size, reference_domain_size)
        data[str(domain_size)] = eta1

    #Save to file
    np.savez_compressed(cdklm_filename, **data)
    
    
kp07_filename = imgdir + "/KP07_data.npz"
benchmarkCDKLM(kp07_filename)

In [ ]:
domain_sizes = []
error = []

with np.load(kp07_filename) as npzfile:
    #Get all domain sizes
    domain_sizes = np.array(npzfile.keys(), dtype=np.int)
    error = np.zeros(len(domain_sizes)-1)
    
    #Get largest domain size
    reference_domain_size = np.max(domain_sizes)
    
    #Remove largest from all domain sizes
    domain_sizes = np.delete(domain_sizes, np.where(domain_sizes == reference_domain_size))
    
    #Get reference
    eta1_ref_disk = npzfile[str(reference_domain_size)].astype(np.float64)
    print(eta1_ref_disk.shape)
    
    #Loop over all other domain sizes
    for l, domain_size in enumerate(domain_sizes):
        #Get simulation
        print(domain_size)
        eta1 = npzfile[str(domain_size)].astype(np.float64)
        
        #Downsample reference to simulation size
        eta1_ref = average(eta1_ref_disk.copy(), reference_domain_size/domain_size)
        
        plt.figure()
        plt.imshow(eta1 - eta1_ref)
        plt.colorbar()
        #eta1_ref_downsampled = eta1_ref[::downsampling, ::downsampling]
        error[l] = np.linalg.norm((eta1_ref - eta1).flatten(), ord=1) / (domain_size*domain_size)
        print(error[l])
        
fig = plt.figure(figsize=(12, 8))
setBwStyles(fig.gca())
plt.loglog(domain_sizes, error)
plt.gca().axis('equal')